# Kreuzbandrisse im Spitzenfussball

In [3]:
#Importieren aller nötigen Bibliotheken

import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
from datetime import timedelta
import datetime as dt
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

## Einlesen aller Spieler in ausgewählten Ligen Europas seit 2011
Alle Spieler, die seit der Saison 2008/09 in einer der folgenden 20 Ligen tätig waren, werden von der Seite www.transfermarkt.ch eingelesen:

Top-5-Ligen (England, Frankreich, Italien, Deutschland, Spanien)

Zweithöchste Ligen der Top-5-Länder

Schweizer Super- und Challenge-League

Top- und zweite Liga von Belgien, Holland, Österreich, Portugal und Schottland


In [ ]:
 # Liste aller Ligen:
url_ligen = ["bundesliga/startseite/wettbewerb/A1/plus/?saison_id=",
             "2-liga/startseite/wettbewerb/A2/plus/?saison_id=",
             "eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=",
             "keuken-kampioen-divisie/startseite/wettbewerb/NL2/plus/?saison_id=",
             "super-league/startseite/wettbewerb/C1/plus/?saison_id=",
             "challenge-league/startseite/wettbewerb/C2/plus/?saison_id=",
             "laliga/startseite/wettbewerb/ES1/plus/?saison_id=",
             "segunda-division/startseite/wettbewerb/ES2/plus/?saison_id=",
             "1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=",
             "2-bundesliga/startseite/wettbewerb/L2/plus/?saison_id=",
             "ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=",
             "ligue-2/startseite/wettbewerb/FR2/plus/?saison_id=",
             "serie-a/startseite/wettbewerb/IT1/plus/?saison_id=",
             "serie-b/startseite/wettbewerb/IT2/plus/?saison_id=",
             "premier-league/startseite/wettbewerb/GB1/plus/?saison_id=",
             "championship/startseite/wettbewerb/GB2/plus/?saison_id=",
             "jupiler-pro-league/startseite/wettbewerb/BE1/plus/?saison_id=",
             "proximus-league/startseite/wettbewerb/BE2/plus/?saison_id=",
             "scottish-premiership/startseite/wettbewerb/SC1/plus/?saison_id=",
             "scottish-championship/startseite/wettbewerb/SC2/plus/?saison_id=",
             "liga-nos/startseite/wettbewerb/PO1/plus/?saison_id=",
             "liga-pro/startseite/wettbewerb/PO2/plus/?saison_id="]
teamurl_liste = []
url_liste = []
url_spielerliste = []

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#durch alle Ligen und Saisons iterieren:

for urll in url_ligen:
    for jahr in range(2011,2019):
        webseite = "https://www.transfermarkt.ch/" + urll + str(jahr)
        r = requests.get(webseite, headers=headers)

        # html-code der Liga- und Saison-url auslesen:
        soup = BeautifulSoup(r.text, 'html.parser') 

        # pro Saison und Liga nach den Club-Webseiten suchen und diese in Liste ablegen
        club_urls_roh = soup.find_all('td', {'class':'hauptlink no-border-links show-for-small show-for-pad'})
        
        teamurl_liste_roh = str(club_urls_roh).split('href=')
        teamurl_liste_roh.pop(0)
        
        #Jede Club-Liste durchiterieren und alle Spieler auswählen, in Liste packen 
        for teamurl_roh in teamurl_liste_roh:
            teamurl = "https://www.transfermarkt.ch" + teamurl_roh.split(" ")[0][:-1][1:]       
            #teamname = teamurl.split(">")[1][:-3]

            #r = requests.get(reinurl,headers=headers)
            print(teamurl)
            #soupverein = BeautifulSoup(r.text, 'html.parser')

            r = requests.get(teamurl, headers=headers)

            soupspieler = BeautifulSoup(r.text, 'html.parser') 

            spielerinfos = str(soupspieler).split('zentriert rueckennummer')
            spielerinfos[len(spielerinfos) - 1] = spielerinfos[len(spielerinfos) - 1][:2500]
            spielerinfos.pop(0)

            for spieler_url_roh in spielerinfos:
                spieler_url = "https://www.transfermarkt.ch" + spieler_url_roh.split("id=")[1].split("href=")[1][1:][:-2].replace("profil","verletzungen")

                url_liste.append(spieler_url)
        
        #Mehrfachnennungen von Spielern streichen
        url_spielerliste = list(set(url_liste))

        teamurl_liste.append(teamurl)
        print(urll + " mit " + str(len(url_liste)) + " Spielern")   




In [37]:
# Wieviele verschiedene Spieler waren in den berücksichtigten Ligen tätig?
len(url_spielerliste)

36650

In [ ]:
# Spielerliste abspeichern in csv-Datei
pd.DataFrame(url_spielerliste).to_csv( "AlleSpieler_Europa_seit 2011.csv")

In [ ]:
url_spielerliste = pd.read_csv("AlleSpieler_Europa_seit 2011.csv")
del url_spielerliste['Unnamed: 0']

## Suchen aller Spieler mit Kreuzbandriss

Jedes der 36650 Spieler-Verletzungsprofile nach Kreuzbandrissen untersuchen.

In [ ]:
url_kreuzband_spielerliste = []
i = 0
cruciatezähler = 0
# alle urls durchiterieren und auf der Spieler-Verletzungsseite schauen, ob's irgendwann Kreuzbandriss gab

for url in url_spielerliste["0"]:
    #url = url.replace("https","http")
    r = requests.get(url, headers=headers)
    print(url + " " + str(i))
    i = i + 1
    soupleistungsdaten = BeautifulSoup(r.text, 'html.parser')
    
    if "Kreuzbandriss" in str(soupleistungsdaten) | "cruciate" in str(soupleistungsdaten):
           
        url_kreuzband_spielerliste.append(url)
        print(url + "  Kreuzbandriss") 

url_kreuzband_spielerliste      

In [262]:
# Wieviele Kreuzbandrisse? (mit Rissen vor 2011!)
len(url_kreuzband_spielerliste)

1636

In [264]:
# Liste mit Kreuzbandriss-Spielern in csv-Datei abspeichern
pd.DataFrame(url_kreuzband_spielerliste).to_csv("Spieler-Urls Kreuzbandverletzte in Europäischen Ligen")

In [ ]:
url_kreuzband_spielerliste = pd.read_csv("Spieler-Urls Kreuzbandverletzte in Europäischen Ligen")
del url_kreuzband_spielerliste['Unnamed: 0']
url_kreuzband_spielerliste

## Daten aller Kreuzbandriss-Spieler auslesen

Für jeden der 1636 Kreuzbandriss-Spieler Daten wie Team, Alter, Verletzungszeit oder Position suchen.

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
riss_anzahl = 0
verletzung = []
verletzungsliste = []
verletztenliste_test = [] #verletztenliste = []


# alle Webseiten der Spieler mit Kreuzbandriss durchsuchen:

for url in url_kreuzband_spielerliste_version2:
    name = ""
    team = ""
    gebdatum = ""
    anfangsdatum = ""
    enddatum = ""
    alter = 0
    fehltage = 0
    saison = ""
    liga = ""
    position = ""
    cruciatezähler = 0
    
    r = requests.get(url, headers=headers)
    print(url)
    soupleistungsdaten = BeautifulSoup(r.text, 'html.parser')
        
    if "cruciate" in str(soupleistungsdaten):
        cruciatezähler = cruciatezähler + 1
        print ("cruciate: " + url)
    if "Kreuzbandriss" in str(soupleistungsdaten):
        namensuppe = str(soupleistungsdaten).split("itemprop=\"name\"")
        vorname = namensuppe[1].split("<b>")[0][1:][:-1]
        nachname = namensuppe[1].split("<b>")[1].split("</b>")[0]
        name = vorname + " " + nachname
        gebdatumstr = re.search(r'\d{2}\.\d{2}\.\d{4}', str(soupleistungsdaten).split("birthDate")[1])
        print(gebdatumstr)
        gebdatum = datetime.strptime(gebdatumstr.group(), '%d.%m.%Y').date()
        try:
            position = str(soupleistungsdaten).split(">Position:</span>\n<span class=\"dataValue\">\n\t\t\t\t\t\t\t\t")[1].split("</span")[0]
        except: continue
        print(name)
        for anzahlrisse in range (1,4):
            try:
                anfangsdatum = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse].split("zentriert bg_rot_20\">")[1][:10]
                saison = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse - 1][-70:].split("zentriert bg_rot_20\">")[1][:2]
                match = re.search(r'\d{2}\.\d{2}\.\d{4}', anfangsdatum)
                anfangsdatum = datetime.strptime(match.group(), '%d.%m.%Y').date()
                
                rissdatum_komma = anfangsdatum.year + ((anfangsdatum.month - 1) * 30 + anfangsdatum.day)/365

                team = str(soupleistungsdaten).split("Kreuzbandriss")[1].split("img alt=\"")[1].split("\"")[0]
                print("noch verletzt")
            except:

                try:
                    anfangsdatum = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse].split("zentriert\">")[1][:10]
                    saison = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse - 1][-50:].split("zentriert\">")[1][:2]
                    enddatum = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse].split("zentriert\">")[2][:10]
                    fehltage = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse].split("rechts\">")[1][:3]
                    fehlspiele = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse].split("wappen_verletzung\">")[1][:2]
                    team = str(soupleistungsdaten).split("Kreuzbandriss")[anzahlrisse].split("img alt=\"")[1].split("\"")[0]

                    match = re.search(r'\d{2}\.\d{2}\.\d{4}', anfangsdatum)
                    anfangsdatum = datetime.strptime(match.group(), '%d.%m.%Y').date()
                    
                    rissdatum_komma = anfangsdatum.year + ((anfangsdatum.month - 1) * 30 + anfangsdatum.day)/365

                    try:
                        match = re.search(r'\d{2}\.\d{2}\.\d{4}', enddatum)
                        enddatum = datetime.strptime(match.group(), '%d.%m.%Y').date()
                    except: print("noch verletzt")
                
                except: continue
            alterbeiriss = relativedelta(anfangsdatum, gebdatum)
            alter = alterbeiriss.years
                
            url_einzelspiele = url.replace("verletzungen","leistungsdaten") + "/plus/0?saison=20" + saison
            r = requests.get(url_einzelspiele, headers=headers)
            soupeinzelspiele = BeautifulSoup(r.text, 'html.parser')
            liga = str(soupeinzelspiele).split(str(saison) + "/spieltag")[0][-70:].split("/")[1]

            # alle herausgefilterten Daten in einem Dictionary abspeichern
            
            verletztendaten = ({"Name":name, 
                                "Verein":team, 
                                "Geburt":gebdatum, 
                                "Rissdatum":anfangsdatum, 
                                "Rissdatum dezimal":rissdatum_komma,
                                "Alter bei Riss":alter, 
                                "Verletzungszeit":fehltage,
                                "Position":position,
                                "Saison":saison, 
                                "Liga":liga, 
                                "Webseite":url})
            verletztenliste.append(verletztendaten)
            

In [ ]:
# komplette Datensätze der Kreuzbanriss-Spieler in csv-Datei abspeichern
pd.DataFrame(verletztenliste).to_csv( "Kreuzband_Europa.csv")


In [257]:
dfeuro = pd.read_csv("Kreuzband_Europa.csv")

# allfällige Duplikate löschen
dfeuro = dfeuro.drop_duplicates(subset=['Name', 'Rissdatum'])
len(dfeuro)

1851

In [258]:
#Anpasssungen bei Lesefehlern
dfeuro.loc[dfeuro.Verein == "FC St. Gallen 1879", "Liga"] = "super-league"
dfeuro.loc[dfeuro.Name == "Matias Vitkieviez", "Verein"] = "BSC Young Boys"
dfeuro.loc[dfeuro.Name == "Theofanis Gekas", "Liga"] = "türkei"

In [259]:
# erste Spalte löschen
del dfeuro['Unnamed: 0']
# mehr Reihen anzeigen
pd.set_option('display.max_rows', 1000)
# Grund-Dataframe sichern
dfeurobasis = dfeuro

### Wieviele Kreuzbandrisse insgesamt?

In [260]:
#bereinigter Datensatz ohne Kreuzbandrisse vor der Saison 2010/09 und nach der Saison 2018/19
dfeuro = dfeuro[(dfeuro["Saison"] < 19)]
dfeuro = dfeuro[(dfeuro["Saison"] > 10)]
len(dfeuro)

1430

### Wie viele doppelte Risse?

In [261]:
dfdoppelte = dfeuro.groupby('Name').count()
dfdoppelte = dfdoppelte[dfdoppelte.Geburt != 1]
len(dfdoppelte)

139

### Wie viele Risse über alle Ligen in den Saisons?

In [ ]:
#Anzahl Risse über alle Ligen
dfeuro.groupby('Saison').count()

In [275]:
# nur die Kreuzbandrisse in gesuchten Ligen behalten
dfeuro = dfeuro[(dfeuro["Liga"] == "1-bundesliga") | (dfeuro["Liga"] == "2-bundesliga") | (dfeuro["Liga"] == "bundesliga") | (dfeuro["Liga"] == "2-liga")| (dfeuro["Liga"] == "challenge-league")| (dfeuro["Liga"] == "super-league")| (dfeuro["Liga"] == "championship")| (dfeuro["Liga"] == "eredivisie")| (dfeuro["Liga"] == "jupiler-pro-league")| (dfeuro["Liga"] == "laliga")| (dfeuro["Liga"] == "laliga2")| (dfeuro["Liga"] == "ligue-1")| (dfeuro["Liga"] == "ligue-2")| (dfeuro["Liga"] == "premier-league")| (dfeuro["Liga"] == "scottish-premiership")| (dfeuro["Liga"] == "serie-a") | (dfeuro["Liga"] == "serie-b") | (dfeuro["Liga"] == "keuken-kampionen-divisie")]
len(dfeuro)

883

In [276]:
dfeuro_ohne_zweifachrisse = dfeuro.drop_duplicates(subset=['Name', 'Geburt'])

len(dfeuro_ohne_zweifachrisse)

792

In [277]:
#finde alle, die zwei oder mehr Kreuzbandrisse hatten
dfdoppelte = dfeuro.groupby('Name').count()
dfdoppelte = dfdoppelte[dfdoppelte.Geburt != 1]
len(dfdoppelte)

86

In [ ]:
dfdoppelte = dfeuro[dfeuro.duplicated(['Name'], keep=False)]
dfdoppelte


In [279]:
dfeuro.groupby('Liga').count().sort_values

<bound method DataFrame.sort_values of                       Alter bei Riss  Geburt  Name  Position  Rissdatum  \
Liga                                                                      
1-bundesliga                      64      64    64        64         64   
2-bundesliga                      83      83    83        83         83   
2-liga                            52      52    52        52         52   
bundesliga                        68      68    68        68         68   
challenge-league                  38      38    38        38         38   
championship                      23      23    23        23         23   
eredivisie                        61      61    61        61         61   
jupiler-pro-league                31      31    31        31         31   
laliga                            83      83    83        83         83   
laliga2                           78      78    78        78         78   
ligue-1                           50      50    50        50 

In [281]:
dfeuro.Liga = dfeuro.Liga.replace({"premier-league": "Premier League"})
dfeuro.Liga = dfeuro.Liga.replace({"super-league": "Super League"})
dfeuro.Liga = dfeuro.Liga.replace({"challenge-league": "Challenge League"})
dfeuro.Liga = dfeuro.Liga.replace({"ligue-1": "Ligue 1"})
dfeuro.Liga = dfeuro.Liga.replace({"ligue-2": "Ligue 2"})
dfeuro.Liga = dfeuro.Liga.replace({"serie-a": "Serie A"})
dfeuro.Liga = dfeuro.Liga.replace({"serie-b": "Serie B"})
dfeuro.Liga = dfeuro.Liga.replace({"scottish-premiership": "Scottish Premier"})
dfeuro.Liga = dfeuro.Liga.replace({"jupiler-pro-league": "Belgien: Jupiler League"})
dfeuro.Liga = dfeuro.Liga.replace({"1-bundesliga": "1. Bundesliga"})
dfeuro.Liga = dfeuro.Liga.replace({"2-bundesliga": "2. Bundesliga"})
dfeuro.Liga = dfeuro.Liga.replace({"bundesliga": "Bundesliga Österreich"})
dfeuro.Liga = dfeuro.Liga.replace({"laliga": "La Liga"})
dfeuro.Liga = dfeuro.Liga.replace({"eredivisie": "Eredivisie"})

## Wie viele Risse gab es in welchen Ligen? (pro Team und Saison)

In [85]:
dfpremierleague = dfeuro[(dfeuro["Liga"] == "Premier League")]

print ("In der Premier League gab es " + str(len(dfpremierleague)/20/8) + " Kreuzbandrisse pro Saison und Club")


In der Premier League gab es 0.3875 Kreuzbandrisse pro Saison und Club


In [86]:
dfsuperleague = dfeuro[(dfeuro["Liga"] == "Super League")]

print ("In der Super League gab es " + str(len(dfsuperleague)/10/8) + " Kreuzbandrisse pro Saison und Club")


In der Super League gab es 0.6875 Kreuzbandrisse pro Saison und Club


In [87]:
dfbundesliga_ö = dfeuro[(dfeuro["Liga"] == "Bundesliga Österreich")]

print ("In der österreichischen Bundesliga gab es " + str(len(dfbundesliga_ö)/10.25/8) + " Kreuzbandrisse pro Saison und Club")


In der österreichischen Bundesliga gab es 0.8292682926829268 Kreuzbandrisse pro Saison und Club


In [88]:
dflaliga = dfeuro[(dfeuro["Liga"] == "La Liga")]

print ("In der La Liga gab es " + str(len(dflaliga)/20/8) + " Kreuzbandrisse pro Saison und Club")


In der La Liga gab es 0.51875 Kreuzbandrisse pro Saison und Club


In [89]:
dfseriea = dfeuro[(dfeuro["Liga"] == "Serie A")]

print ("In der Serie A gab es " + str(len(dfseriea)/20/8) + " Kreuzbandrisse pro Saison und Club")

In der Serie A gab es 0.375 Kreuzbandrisse pro Saison und Club


In [90]:
dfligue1 = dfeuro[(dfeuro["Liga"] == "Ligue 1")]

print ("In der Ligue 1 gab es " + str(len(dfligue1)/20/8) + " Kreuzbandrisse pro Saison und Club")

In der Ligue 1 gab es 0.3125 Kreuzbandrisse pro Saison und Club


In [91]:
df1bundesliga = dfeuro[(dfeuro["Liga"] == "1. Bundesliga")]

print ("In der deutschen Bundesliga gab es " + str(len(df1bundesliga)/18/8) + " Kreuzbandrisse pro Saison und Club")

In der deutschen Bundesliga gab es 0.4444444444444444 Kreuzbandrisse pro Saison und Club


In [92]:
dferedivisie = dfeuro[(dfeuro["Liga"] == "Eredivisie")]

print ("In der Eredivisie gab es " + str(len(dferedivisie)/18/8) + " Kreuzbandrisse pro Saison und Club")

In der Eredivisie gab es 0.4236111111111111 Kreuzbandrisse pro Saison und Club


In [93]:
dfjupiler = dfeuro[(dfeuro["Liga"] == "Jupiler")]

print ("In der Jupiler pro League Belgiens gab es " + str(len(dferedivisie)/16/8) + " Kreuzbandrisse pro Saison und Club")

In der Jupiler pro League Belgiens gab es 0.4765625 Kreuzbandrisse pro Saison und Club


In [94]:
dfgewisseliga = dfeuro[(dfeuro["Verein"] == "FC Basel 1893")]

print ("In diesem Verein gab es " + str(len(dfgewisseliga)) + " Kreuzbandrisse")


In diesem Verein gab es 4 Kreuzbandrisse


In [97]:
# Spitzenreiter FC St.Gallen 2018/19: Kein anderer Verein hatte in der vergangenen Saison gleich vier Kreuzbandrisse

df2019alle = dfeuro[dfeuro["Saison"] == 18]
df2019alle.groupby('Verein').count().sort_values

<bound method DataFrame.sort_values of                              Alter bei Riss  Geburt  Liga  Name  Position  \
Verein                                                                      
1.FC Köln                                 1       1     1     1         1   
1.FC Nürnberg                             1       1     1     1         1   
AC Florenz                                1       1     1     1         1   
AC Mailand                                1       1     1     1         1   
AFC Bournemouth                           2       2     2     2         2   
AS Monaco                                 1       1     1     1         1   
AS Saint-Étienne                          1       1     1     1         1   
Arminia Bielefeld                         1       1     1     1         1   
Atalanta Bergamo                          1       1     1     1         1   
Atlético Madrid B                         1       1     1     1         1   
BSC Young Boys                       

In [98]:
# Spitzenreiter FC Zürich: Kein anderer Verein hatte in den vergangenen acht Jahren mehr Kreuzbandrisse

df2019alle = dfeuro[dfeuro["Saison"] > 10]
df2019alle.groupby('Verein').count().sort_values

<bound method DataFrame.sort_values of                               Alter bei Riss  Geburt  Liga  Name  Position  \
Verein                                                                       
1.FC Heidenheim 1846                       1       1     1     1         1   
1.FC Kaiserslautern                        4       4     4     4         4   
1.FC Köln                                  5       5     5     5         5   
1.FC Nürnberg                              4       4     4     4         4   
1.FC Nürnberg II                           1       1     1     1         1   
1.FC Union Berlin                          2       2     2     2         2   
1.FC Union Berlin II                       1       1     1     1         1   
1.FSV Mainz 05                             3       3     3     3         3   
AC Ajaccio                                 2       2     2     2         2   
AC Florenz                                 2       2     2     2         2   
AC Le Havre              

In [52]:
# Kreuzbandrisse im Jahr 2019 in der Super League
df2019sg = dfeuro[dfeuro["Liga"] == "Super League"]
df2019sg = df2019sg[df2019sg["Rissdatum dezimal"] > 2019]
df2019sg

,Alter bei Riss,Geburt,Liga,Name,Position,Rissdatum,Rissdatum dezimal,Saison,Verein,Verletzungszeit,Webseite
743,32,1986-07-08,Super League,Adrian Winter,Rechtes Mittelfeld,2019-03-10,2019.19,18,FC Zürich,201,https://www.transfermarkt.ch/adrian-winter/ver...
952,28,1990-08-21,Super League,Alain Wiss,Innenverteidiger,2019-01-17,2019.05,18,FC St. Gallen 1879,202,https://www.transfermarkt.ch/alain-wiss/verlet...
961,26,1993-02-12,Super League,Hekuran Kryeziu,Defensives Mittelfeld,2019-04-22,2019.31,18,FC Zürich,0,https://www.transfermarkt.ch/hekuran-kryeziu/v...
1031,19,1999-10-23,Super League,Nino Ziswiler,Torwart,2019-04-29,2019.33,18,FC Thun,0,https://www.transfermarkt.ch/nino-ziswiler/ver...
1390,22,1997-01-17,Super League,Musah Nuhu,Innenverteidiger,2019-06-06,2019.43,18,FC St. Gallen 1879,0,https://www.transfermarkt.ch/musah-nuhu/verlet...
1662,32,1986-09-22,Super League,Dennis Hediger,Defensives Mittelfeld,2019-02-11,2019.11,18,FC Thun,0,https://www.transfermarkt.ch/dennis-hediger/ve...
1826,22,1996-10-25,Super League,Sandro Lauper,Zentrales Mittelfeld,2019-05-26,2019.40,18,BSC Young Boys,0,https://www.transfermarkt.ch/sandro-lauper/ver...


## Wieviele Kreuzbandrisse gab es pro Saison in europäischen Ligen?

In [294]:
dfsaison = dfeuro[dfeuro["Saison"] == 18]
dfsaison = dfsaison[dfsaison["Liga"] == "Premier League"]
len(dfsaison)

10

In [302]:
dfsaison = dfeuro[(dfeuro["Liga"] == "Super League") | (dfeuro["Liga"] == "1. Bundesliga") | (dfeuro["Liga"] == "Premier League") | (dfeuro["Liga"] == "La Liga") | (dfeuro["Liga"] == "Ligue 1") | (dfeuro["Liga"] == "Serie A")]
dfsaison = dfsaison[dfsaison["Saison"] == 18]
len(dfsaison)

62

## Wie lange ist die Verletzungszeit in den Ligen?

In [116]:

dfeuro.Verletzungszeit = pd.to_numeric(dfeuro.Verletzungszeit, errors='coerce')

# alle derzeit Verletzten abziehen
dfeuro0 = dfeuro[dfeuro["Verletzungszeit"] > 0]
len(dfeuro0)

792

In [143]:
dfeurosaison = dfeuro0[dfeuro0["Saison"] == 18]
#Ausreisser von über 400 und weniger als 120 löschen
dfeurosaison = dfeurosaison[(dfeurosaison["Verletzungszeit"] > 120) & (dfeurosaison["Verletzungszeit"] < 400)]
dfeurosaison["Verletzungszeit"].mean()

221.50666666666666

In [154]:
dfeurosaison = dfeuro0[dfeuro0["Liga"] == "Super League"]
#Ausreisser von über 400 und weniger als 120 löschen
dfeurosaison = dfeurosaison[(dfeurosaison["Verletzungszeit"] > 120) & (dfeurosaison["Verletzungszeit"] < 400)]
dfeurosaison["Verletzungszeit"].median()

237.0

In [200]:
dfeurosaison = dfeuro0[(dfeuro0["Liga"] == "1. Bundesliga") & (dfeuro0["Saison"] == 17)]
#Ausreisser von über 400 und weniger als 120 löschen
dfeurosaison = dfeurosaison[(dfeurosaison["Verletzungszeit"] > 120) & (dfeurosaison["Verletzungszeit"] < 400)]
dfeurosaison["Verletzungszeit"].mean()

240.55555555555554

In [208]:
dfeurosaison = dfeuro0[dfeuro0["Liga"] == "Premier League"]
dfeurosaison = dfeurosaison[dfeurosaison["Saison"] == 17]
#Ausreisser von über 400 und weniger als 120 löschen
dfeurosaison = dfeurosaison[(dfeurosaison["Verletzungszeit"] > 120) & (dfeurosaison["Verletzungszeit"] < 400)]
dfeurosaison["Verletzungszeit"].mean()

250.57142857142858

In [290]:
dfeurosaison_sl = dfeuro0[(dfeuro0["Saison"] == 9) & (dfeuro0["Liga"] == "superleague")]
dfeurosaison_sl["Verletzungszeit"].mean()

210.0

## In welchen Ligen kommen die Spieler am schnellsten zurück?

In [47]:
dfeuro0[dfeuro0["Liga"] == "superleague"]["Verletzungszeit"].median()

nan

In [294]:
dfeuro0[dfeuro0["Liga"] == "premierleague"]["Verletzungszeit"].median()

236.0

In [295]:
dfeuro0[dfeuro0["Liga"] == "seriea"]["Verletzungszeit"].median()

181.0

In [296]:
dfeuro0[dfeuro0["Liga"] == "laliga"]["Verletzungszeit"].median()

190.0

In [297]:
dfeuro0[dfeuro0["Liga"] == "1bundesliga"]["Verletzungszeit"].median()

210.0

In [298]:
dfeuro0[dfeuro0["Liga"] == "ligue1"]["Verletzungszeit"].median()

184.0

In [299]:
dfeuro0[dfeuro0["Liga"] == "bundesliga"]["Verletzungszeit"].median()

197.0